# 📊 ETL, EDA y Predicción – Curtiduría Tungurahua S.A.
### Versión Detallada (Opción B)

Este notebook está diseñado **para personas sin conocimientos técnicos**, explicando cada transformación, limpieza, gráfica y predicción paso a paso.

Se separan claramente las etapas:
- **Transformación de datos (tipos)**
- **Limpieza de datos (errores, vacíos, incoherencias)**
- **EDA con gráficas explicadas**
- **Predicción sencilla**

## 1️⃣ Importación de librerías

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
%matplotlib inline

## 2️⃣ Cargar los datos

Aquí cargamos el archivo **Produccion-Curtiduria.csv**, que contiene los datos simulados de la producción de la empresa.

In [ ]:
df = pd.read_csv("Produccion-Curtiduria.csv")
df.head()

## 3️⃣ Transformación de datos (Tipos de datos)
En esta etapa **no limpiamos errores**, solo transformamos el tipo de datos.
Explicamos columna por columna **qué tipo tenía y a qué tipo lo convertimos**.

### 🔹 3.1 Transformación de columnas tipo Fecha

➡️ **`Fecha_Producción`**
- Tipo original: *texto (string)*
- Problema: las computadoras no saben que es una fecha, solo ven texto.
- Solución: convertirla al tipo **datetime**, que permite ordenar, filtrar y agrupar.


In [ ]:
df['Fecha_Producción'] = pd.to_datetime(df['Fecha_Producción'], errors='coerce')
df['Fecha_Producción'].head()

### 🔹 3.2 Transformación de columnas numéricas – Tipo Entero (Int64)

Estas columnas representan cantidades contadas. Por eso deben ser **enteros (Int64)**.

Columnas:
- `Unidades_Producidas`
- `Unidades_Defectuosas`
- `Unidades_Aprobadas`
- `Duración_Proceso_Min`

➡️ Originalmente venían como texto.
➡️ Convertimos a **Int64**, que soporta valores nulos sin errores.

In [ ]:
cols_int = ['Unidades_Producidas','Unidades_Defectuosas','Unidades_Aprobadas','Duración_Proceso_Min']

for col in cols_int:
    df[col] = pd.to_numeric(df[col], errors='coerce').astype('Int64')

df[cols_int].head()

### 🔹 3.3 Transformación de columnas numéricas – Tipo Decimal (float)

Estas columnas requieren decimales porque representan porcentajes, costos o valores energéticos.

Columnas:
- `Rendimiento_%`
- `Costo_Unitario`
- `Costo_Total`
- `Consumo_Energía_kWh`

➡️ Se convierten a **float** para permitir cálculos precisos.

In [ ]:
cols_float = ['Rendimiento_%','Costo_Unitario','Costo_Total','Consumo_Energía_kWh']

for col in cols_float:
    df[col] = pd.to_numeric(df[col], errors='coerce')

df[cols_float].head()

### 🔹 3.4 Transformación de columnas categóricas

Estas columnas contienen texto repetido (como 'Turno A', 'Turno B').
➡️ Convertimos a **category**, un tipo eficiente para texto repetitivo.

Columnas:
- `Turno`, `Línea`, `Máquina`, `Producto`, `Estado_Producción`, `Supervisor`

In [ ]:
categorical_cols = ['Turno','Línea','Máquina','Producto','Estado_Producción','Supervisor']

for col in categorical_cols:
    df[col] = df[col].astype('category')

df[categorical_cols].head()

## 4️⃣ Limpieza de Datos (Separado de transformación)
Aquí revisamos valores faltantes, incoherentes o imposibles.
No modificamos tipos: solo limpiamos.

### 🔍 4.1 Buscar valores faltantes

In [ ]:
df.isna().sum()

Si encontramos valores faltantes en datos críticos, podemos:
- Rellenar
- Eliminar
- Sustituir con promedios
En este dataset no se aplica limpieza agresiva porque es un archivo simulado.

### ➕ 4.2 Nueva columna: Fecha sin hora

In [ ]:
df['Fecha_Día'] = df['Fecha_Producción'].dt.date
df[['Fecha_Producción','Fecha_Día']].head()

### ➕ 4.3 Nueva columna: Tasa de defectos (%)

In [ ]:
df['Tasa_Defectos_%'] = (df['Unidades_Defectuosas'] / df['Unidades_Producidas']) * 100
df[['Unidades_Producidas','Unidades_Defectuosas','Tasa_Defectos_%']].head()

## 5️⃣ EDA – Análisis Exploratorio de Datos
Las siguientes gráficas tienen explicaciones para personas sin experiencia técnica.

### 📈 5.1 Producción diaria

In [ ]:
prod_dia = df.groupby('Fecha_Día')['Unidades_Aprobadas'].sum().reset_index()
prod_dia['Fecha_Día'] = pd.to_datetime(prod_dia['Fecha_Día'])

plt.figure(figsize=(10,5))
plt.plot(prod_dia['Fecha_Día'], prod_dia['Unidades_Aprobadas'])
plt.title("Producción diaria de unidades aprobadas")
plt.xlabel("Fecha")
plt.ylabel("Unidades aprobadas")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

### 🧑‍🏭 5.2 Producción por turno

In [ ]:
turno = df.groupby('Turno')['Unidades_Aprobadas'].sum().reset_index()

plt.figure(figsize=(6,4))
plt.bar(turno['Turno'], turno['Unidades_Aprobadas'])
plt.title("Producción por turno")
plt.xlabel("Turno")
plt.ylabel("Unidades aprobadas")
plt.tight_layout()
plt.show()

### ➕ 5.3 Gráficas adicionales: análisis profundo

#### 🔵 Producción vs Defectos

In [ ]:
plt.figure(figsize=(7,5))
plt.scatter(df['Unidades_Producidas'], df['Unidades_Defectuosas'])
plt.title("Producción vs Defectos")
plt.xlabel("Unidades producidas")
plt.ylabel("Unidades defectuosas")
plt.tight_layout()
plt.show()

#### ⚡ Consumo energético por línea

In [ ]:
energia = df.groupby('Línea')['Consumo_Energía_kWh'].mean().reset_index()

plt.figure(figsize=(8,5))
plt.bar(energia['Línea'], energia['Consumo_Energía_kWh'])
plt.title("Consumo energético promedio por línea")
plt.xlabel("Línea")
plt.ylabel("kWh")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

#### 🎯 Rendimiento vs Consumo Energético

In [ ]:
plt.figure(figsize=(7,5))
plt.scatter(df['Rendimiento_%'], df['Consumo_Energía_kWh'])
plt.title("Rendimiento vs Consumo Energético")
plt.xlabel("Rendimiento (%)")
plt.ylabel("Consumo energético (kWh)")
plt.tight_layout()
plt.show()

## 6️⃣ Predicción de la producción

Se utiliza un modelo **muy sencillo** para estimar la tendencia futura.

In [ ]:
data_pred = prod_dia.copy().reset_index(drop=True)
data_pred['t'] = np.arange(len(data_pred))

X = data_pred[['t']]
y = data_pred['Unidades_Aprobadas']

modelo = LinearRegression()
modelo.fit(X, y)

dias = 7
t_fut = np.arange(len(data_pred), len(data_pred)+dias).reshape(-1,1)
pred = modelo.predict(t_fut)

fecha_fin = data_pred['Fecha_Día'].max()
fechas_fut = pd.date_range(start=fecha_fin + pd.Timedelta(days=1), periods=dias)

pred_df = pd.DataFrame({'Fecha_Día': fechas_fut, 'Predicción': pred})

plt.figure(figsize=(10,5))
plt.plot(data_pred['Fecha_Día'], data_pred['Unidades_Aprobadas'], label="Histórico")
plt.plot(pred_df['Fecha_Día'], pred_df['Predicción'], marker='o', linestyle="--", label="Predicción")
plt.title("Predicción de producción (7 días)")
plt.xlabel("Fecha")
plt.ylabel("Unidades")
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 🏁 7️⃣ Conclusiones